# Deprecate Unused Tables

This notebook provides an example for getting all of the unused tables in Snowflake. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update the variable. For self hosted, you need to add /api/v1 to the end of the SECODA_API_URL, i.e, https://secoda.paystack.com/api/v1

In [22]:

import requests

API_KEY = ""
SECODA_API_URL = "https://api.secoda.co"

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))


def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

In [20]:
import dateutil.parser
from datetime import datetime, timedelta
import pytz

thirty_days_ago = datetime.utcnow() - timedelta(days=30)

def get_unused_and_stale_tables():
    pages = 1
    page = 1
    unused_tables = []
    stale_tables = []
    while page <= pages:
        print(f'Getting page {page}/{pages}')
        response = session.get(build_url(f"/table/tables?integration_id=a7123601-d1f1-42f0-b8db-1e40ca273a81&page={page}"))
        data = response.json()
        pages = data["total_pages"]
        page += 1
        tables = data.get("results", [])
        for table in tables:
            
            # Check if the table has been queried in last 7 days
            if table.get("external_usage") == 0:
                unused_tables.append(table)
                
            # Check if a table has been updated in last 30 days
            if thirty_days_ago.replace(tzinfo=pytz.UTC) > dateutil.parser.parse(table.get("external_updated_at")).replace(tzinfo=pytz.UTC):
                stale_tables.append(table)

    return unused_tables, stale_tables

unused_tables, stale_tables = get_unused_and_stale_tables()

# Write to a file unused_tables.txt
with open("data/unused_tables.txt", "w") as f:
    for table in unused_tables:
        f.write(f"{table['database']}.{table['schema']}.{table['title']}\n")

with open("data/stale_tables.txt", "w") as f:
    for table in stale_tables:
        f.write(f"{table['database']}.{table['schema']}.{table['title']}\n")



Getting page 1/1
